In [15]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torchmetrics import R2Score
from PIL import Image

# Define custom dataset
class PlantDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_id = str(self.data_frame.iloc[idx, 0])
        img_path = os.path.join(self.root_dir, img_id + '.jpeg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        ancillary_data = self.data_frame.iloc[idx, 1:164].values.astype('float32')
        if 'train' in self.root_dir:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return [ancillary_data, image], labels
        else:
            return [ancillary_data, image], img_id

# Define transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    # No noise needed since clear images
])

# Load the datasets
train_dataset = PlantDataset(csv_file='data/train_preprocessed.csv', root_dir='data/train_images', transform=data_transforms)
test_dataset = PlantDataset(csv_file='data/test_preprocessed.csv', root_dir='data/test_images', transform=data_transforms)

In [16]:
# Create training subset and validation subset
# train_size = len(train_dataset) // 5
# train_subset_indices, _ = train_test_split(list(range(len(train_dataset))), train_size=train_size, random_state=42)
# val_size = int(0.15 * train_size)
# train_indices, val_indices = train_test_split(train_subset_indices, test_size=val_size, random_state=42)

# train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=32, shuffle=True, pin_memory=True)
# val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=32, shuffle=False, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


val_fraction = 1 / 15  # Change to 1/20 if desired
val_size = int(val_fraction * len(train_dataset))

# Split the dataset indices into training and validation
train_indices, val_indices = train_test_split(
    list(range(len(train_dataset))), test_size=val_size, random_state=42)

# Create DataLoaders for training, validation, and test sets
train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=32, shuffle=True, pin_memory=True)
val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=32, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [17]:
print(len(test_dataset))
print(len(test_loader))
print(len(train_loader))
print(len(val_loader))

6391
200
1265
91


In [18]:
class ResNet18_essay(nn.Module):
    def __init__(self):
        super(ResNet18_essay, self).__init__()
        self.base_cnn = models.resnet101(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        self.cnn_output = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )

        self.main_output = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)

In [19]:
# Instantiate the models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [20]:
train_losses = []
val_losses = []
val_r2_scores = []

model = ResNet18_essay().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
r2_score = R2Score(num_outputs=1).to(device)

num_epochs = 30
target_feature = 0

In [21]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for [ancillary_data, images], labels in train_loader:
        ancillary_data, images, labels = ancillary_data.to(device), images.to(device), labels.to(device)
        labels = labels[:,target_feature].unsqueeze(1)

        outputs = model([ancillary_data, images])
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate training loss
        train_loss += loss.item()

    # Record average training loss for this epoch
    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Validation loop
    model.eval()
    val_loss = 0
    val_outputs = []
    val_labels = []
    with torch.no_grad():
        for [ancillary_data, images], labels in val_loader:
            ancillary_data, images, labels = ancillary_data.to(device), images.to(device), labels.to(device)
            labels = labels[:,target_feature].unsqueeze(1)

            outputs = model([ancillary_data, images])
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            val_outputs.append(outputs)
            val_labels.append(labels)

    # Record average validation loss for this epoch
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    # Calculate R² score
    val_outputs = torch.cat(val_outputs)
    val_labels = torch.cat(val_labels)
    r2 = r2_score(val_outputs, val_labels).item()
    val_r2_scores.append(r2)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.8f}, Validation Loss: {val_loss:.8f}, R² Score: {r2:.8f}')
    if epoch + 1 >= 5:
        torch.save(model.state_dict(), f'temp/{target_feature + 1}_epoch_{epoch+1}_resnet101.pth')

Epoch [1/30], Training Loss: 0.08266592, Validation Loss: 0.01412652, R² Score: 0.11115044
Epoch [2/30], Training Loss: 0.01276750, Validation Loss: 0.01343183, R² Score: 0.15321255
Epoch [3/30], Training Loss: 0.00855793, Validation Loss: 0.01400962, R² Score: 0.11580390
Epoch [4/30], Training Loss: 0.00400308, Validation Loss: 0.01522813, R² Score: 0.03878289
Epoch [5/30], Training Loss: 0.00278689, Validation Loss: 0.01442183, R² Score: 0.08861911
Epoch [6/30], Training Loss: 0.00214187, Validation Loss: 0.01384568, R² Score: 0.12700468
Epoch [7/30], Training Loss: 0.00194371, Validation Loss: 0.01402778, R² Score: 0.11519915
Epoch [8/30], Training Loss: 0.00189148, Validation Loss: 0.01377145, R² Score: 0.13024902
Epoch [9/30], Training Loss: 0.00170867, Validation Loss: 0.01371720, R² Score: 0.13517112
Epoch [10/30], Training Loss: 0.00141915, Validation Loss: 0.01322453, R² Score: 0.16624194
Epoch [11/30], Training Loss: 0.00125669, Validation Loss: 0.01335918, R² Score: 0.155838